In [1]:
import pandas as pd 
import numpy as np
from geopy.geocoders import Nominatim
from functools import lru_cache

In [2]:
df = pd.read_csv("policekillings.csv")
df.shape

(9081, 38)

In [3]:
# filter null city values
df = df[(df["City"].notnull()) & (df["State"].notnull())]
df.shape

(9072, 38)

In [4]:
# test city conversions
address='Denver, CO, United States of America'
geolocator = Nominatim(user_agent="Your_Name")
location = geolocator.geocode(address, timeout = None)
print(location.address)
print((location.latitude, location.longitude))

Denver, Denver County, Colorado, United States
(39.7392364, -104.9848623)


In [5]:
# gen coordinates
# memoization table --> geolocator calls limited by api refresh rate
m = {}
bad_adds = []
def gen_coord(row):
    print(len(m))
    address = row["City"] + ", " + row["State"] + ", United States of America" 
    if address in m:
        return m[address]
    else:
        location = geolocator.geocode(address)
        if location:
            m[address] = (location.latitude, location.longitude)
        else:
            print(address)
            bad_adds.append(address)
            m[address] = None
    return m[address]

df["coordinates"] = df[["City", "State"]].apply(gen_coord, axis = 1)
df.head()

26
2626
2626
2627
2627
2627
2627
2628
2629
2630
2630
2631
2631
2631
2631
2631
2631
2631
2632
2633
2633
2634
2634
2635
2635
2636
2637
2637
2638
2639
2639
2639
2639
2639
2640
2640
2641
2642
2642
2642
2643
2643
2644
2645
2645
2646
2647
2648
2648
2648
2648
2648
2648
2649
2650
2651
2652
2653
2653
2653
2653
2653
2653
2653
2654
2654
2654
2654
2655
2655
2655
2655
2656
2657
2658
2658
2658
2658
2659
2659
2659
2659
2660
2660
2660
2660
2660
2660
2660
2660
2660
2660
2661
2662
2662
2663
2664
2664
2664
2664
2664
2665
2665
2665
2666
2666
2667
2667
2668
2668
2668
2669
2669
2669
2670
2670
2671
2672
2673
2674
2674
2674
2674
2674
2674
2674
2675
2675
2675
2675
2675
2675
2675
2675
2675
2675
2676
2677
2677
2677
2677
2678
2678
2679
2680
2680
2680
2680
2680
2680
2681
2681
2682
2682
2682
2683
2683
2683
2684
2684
2684
2684
2684
2684
2684
2684
2684
2685
2685
2685
2686
2686
2686
2686
2686
2687
2688
2689
2689
2690
2691
2691
2691
2691
2691
2691
2691
2692
2692
2692
2693
2693
2694
2694
2695
2695
2695
2695
2696
2696
26

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Encounter Type (DRAFT),Initial Reported Reason for Encounter (DRAFT),Names of Officers Involved (DRAFT),Race of Officers Involved (DRAFT),Known Past Shootings of Officer(s) (DRAFT),Call for Service? (DRAFT),Unnamed: 35,Unnamed: 36,Unnamed: 37,coordinates
0,Name withheld by police,Unknown,Male,Unknown Race,NaN,4/18/21,NaN,Detroit,MI,NaN,...,Part 1 Violent Crime,shooting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(42.3315509, -83.0466403)"
1,Name withheld by police,Unknown,Male,Unknown Race,NaN,4/18/21,NaN,Douglasville,GA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(33.7514966, -84.7477136)"
2,Name withheld by police,Unknown,Male,Unknown Race,NaN,4/18/21,NaN,Fort Worth,TX,NaN,...,Other Non-Violent Offense,carjacking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(32.753177, -97.3327459)"
3,Name withheld by police,Unknown,Male,White,NaN,4/18/21,NaN,Burnsville,MN,NaN,...,Other Non-Violent Offense,carjacking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(44.7670567, -93.2773887)"
4,Name withheld by police,Unknown,Male,Black,NaN,4/17/21,NaN,Winter Haven,FL,NaN,...,Domestic disturbance,Domestic disturbance (armed),NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(28.0222435, -81.7328568)"


In [29]:


print(df["coordinates"][0][0])
df = df[df["coordinates"].notna()]
df["latitude"] = df.apply(lambda row: row["coordinates"][0], axis=1)
df["longitude"] = df.apply(lambda row: row["coordinates"][1], axis=1)

42.3315509


In [31]:
# now that all rows have coordinate positions convert back to csv for use
df.to_csv("police_cleaned.csv", index = False)